# Deep Learning Implementation AssignmentNotebook ini mengimplementasikan jaringan syaraf tiruan untuk memprediksi risiko stroke menggunakan dataset *Stroke Prediction* dari Kaggle. Seluruh komponen utama (aktivasi, loss, forward pass, backward pass, dan algoritma pelatihan) diimplementasikan dari awal dengan `NumPy` untuk memenuhi persyaratan tugas.

## Persiapan Data1. Unduh berkas **`healthcare-dataset-stroke-data.csv`** dari [Stroke Prediction Dataset di Kaggle](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).2. Letakkan berkas tersebut pada direktori root repository ini dan ubah namanya menjadi **`stroke-data.csv`** (atau sesuaikan variabel `DATA_PATH`).3. Jalankan sel berikut untuk memuat dan meninjau data.

In [ ]:
import math
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn-v0_8')
np.random.seed(42)


In [ ]:
DATA_PATH = Path('stroke-data.csv')

if not DATA_PATH.exists():
    raise FileNotFoundError(
        'Dataset tidak ditemukan. Unduh `healthcare-dataset-stroke-data.csv` dari Kaggle, '
        'letakkan sebagai `stroke-data.csv` pada direktori root proyek ini, '
        'lalu jalankan kembali notebook.'
    )

raw_df = pd.read_csv(DATA_PATH)
print(f'Jumlah sampel: {len(raw_df):,}')
raw_df.head()


### Pembersihan & Pra-pemrosesan* Mengisi nilai BMI yang hilang dengan median.* Menghapus kategori gender langka (`Other`).* Melakukan one-hot encoding pada fitur kategorikal.* Menskalakan fitur numerik menggunakan `StandardScaler`.* Membagi data menjadi train dan test dengan proporsi 80:20.

In [ ]:
df = raw_df.copy()

# Tangani nilai yang hilang dan kategori langka
if 'bmi' in df.columns:
    df['bmi'] = df['bmi'].fillna(df['bmi'].median())
if 'gender' in df.columns:
    df = df[df['gender'] != 'Other']

categorical_cols = [
    'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'
]
existing_categoricals = [col for col in categorical_cols if col in df.columns]

df = pd.get_dummies(df, columns=existing_categoricals, drop_first=True)

target_col = 'stroke'
feature_cols = [col for col in df.columns if col != target_col]

X = df[feature_cols].to_numpy(dtype=float)
y = df[target_col].to_numpy(dtype=float).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_dim = X_train.shape[1]
print(f'Jumlah fitur setelah encoding: {input_dim}')
print(f'Proporsi kelas positif di train: {y_train.mean():.3f}')
print(f'Proporsi kelas positif di test : {y_test.mean():.3f}')


## Fungsi Aktivasi dan Loss (From Scratch)Tiga fungsi aktivasi non-linear yang umum digunakan diuji pada hidden layer: **Sigmoid**, **Tanh**, dan **ReLU**. Lapisan output menggunakan sigmoid karena tugasnya adalah klasifikasi biner. Loss function yang dipakai adalah **Binary Cross-Entropy (BCE)**.

In [ ]:
Array = np.ndarray


def sigmoid(z: Array) -> Array:
    return 1.0 / (1.0 + np.exp(-z))


def tanh(z: Array) -> Array:
    return np.tanh(z)


def relu(z: Array) -> Array:
    return np.maximum(0.0, z)


def leaky_relu(z: Array, alpha: float = 0.01) -> Array:
    return np.where(z > 0, z, alpha * z)


def sigmoid_backward(a: Array, _: Array) -> Array:
    return a * (1.0 - a)


def tanh_backward(a: Array, _: Array) -> Array:
    return 1.0 - np.square(a)


def relu_backward(_: Array, z: Array) -> Array:
    return (z > 0).astype(float)


def leaky_relu_backward(_: Array, z: Array, alpha: float = 0.01) -> Array:
    grad = np.ones_like(z) * alpha
    grad[z > 0] = 1.0
    return grad


activation_functions: Dict[str, Dict[str, callable]] = {
    'sigmoid': {'forward': sigmoid, 'backward': sigmoid_backward},
    'tanh': {'forward': tanh, 'backward': tanh_backward},
    'relu': {'forward': relu, 'backward': relu_backward},
    'leaky_relu': {
        'forward': lambda z: leaky_relu(z, alpha=0.01),
        'backward': lambda a, z: leaky_relu_backward(a, z, alpha=0.01),
    },
}


In [ ]:
def binary_cross_entropy(y_true: Array, y_pred: Array, eps: float = 1e-12) -> float:
    y_pred = np.clip(y_pred, eps, 1.0 - eps)
    loss = -(
        y_true * np.log(y_pred) + (1.0 - y_true) * np.log(1.0 - y_pred)
    )
    return float(np.mean(loss))


def binary_cross_entropy_derivative(y_true: Array, y_pred: Array, eps: float = 1e-12) -> Array:
    y_pred = np.clip(y_pred, eps, 1.0 - eps)
    return -(y_true / y_pred) + ((1.0 - y_true) / (1.0 - y_pred))


## Forward Pass, Backward Pass, dan OptimisasiBagian ini mendefinisikan fungsi utilitas untuk:* Inisialisasi bobot (Xavier/Glorot uniform).* Melakukan forward pass.* Melakukan backward pass dan menghitung gradien.* Memperbarui bobot dengan Gradient Descent.* Menghasilkan prediksi probabilitas dan kelas.

In [ ]:
def initialize_parameters(layer_sizes: List[int], rng: np.random.Generator) -> List[Dict[str, Array]]:
    parameters: List[Dict[str, Array]] = []
    for n_in, n_out in zip(layer_sizes[:-1], layer_sizes[1:]):
        limit = math.sqrt(6.0 / (n_in + n_out))
        W = rng.uniform(-limit, limit, size=(n_out, n_in))
        b = np.zeros(n_out)
        parameters.append({'W': W, 'b': b})
    return parameters


def forward_pass(X: Array, params: List[Dict[str, Array]], activation_sequence: List[str]) -> Tuple[Array, List[Dict[str, Array]]]:
    A = X
    caches: List[Dict[str, Array]] = []
    for layer_idx, (layer, activation_name) in enumerate(zip(params, activation_sequence)):
        W, b = layer['W'], layer['b']
        Z = A @ W.T + b
        activation = activation_functions[activation_name]['forward']
        A_next = activation(Z)
        caches.append({'A_prev': A, 'Z': Z, 'A': A_next, 'activation': activation_name})
        A = A_next
    return A, caches


def backward_pass(
    y_true: Array,
    caches: List[Dict[str, Array]],
    params: List[Dict[str, Array]],
    activation_sequence: List[str],
    loss_derivative=binary_cross_entropy_derivative,
) -> List[Dict[str, Array]]:
    grads: List[Dict[str, Array]] = [None] * len(params)
    m = y_true.shape[0]
    dA = loss_derivative(y_true, caches[-1]['A'])

    for idx in reversed(range(len(params))):
        cache = caches[idx]
        activation_name = activation_sequence[idx]
        activation_grad = activation_functions[activation_name]['backward']

        dZ = dA * activation_grad(cache['A'], cache['Z'])
        A_prev = cache['A_prev']
        dW = (dZ.T @ A_prev) / m
        db = np.mean(dZ, axis=0)
        grads[idx] = {'dW': dW, 'db': db}

        if idx != 0:
            dA = dZ @ params[idx]['W']

    return grads


def update_parameters(
    params: List[Dict[str, Array]], grads: List[Dict[str, Array]], learning_rate: float
) -> None:
    for layer, grad in zip(params, grads):
        layer['W'] -= learning_rate * grad['dW']
        layer['b'] -= learning_rate * grad['db']


def predict_proba(
    X: Array, params: List[Dict[str, Array]], activation_sequence: List[str]
) -> Array:
    probs, _ = forward_pass(X, params, activation_sequence)
    return probs


def predict_classes(probs: Array, threshold: float = 0.5) -> Array:
    return (probs >= threshold).astype(int)


In [ ]:
def iterate_batches(
    X: Array,
    y: Array,
    strategy: str,
    batch_size: int,
    rng: np.random.Generator,
) -> Iterable[Tuple[Array, Array]]:
    indices = np.arange(X.shape[0])
    rng.shuffle(indices)

    if strategy == 'batch':
        yield X[indices], y[indices]
    elif strategy == 'stochastic':
        for idx in indices:
            yield X[idx: idx + 1], y[idx: idx + 1]
    elif strategy == 'mini-batch':
        for start in range(0, len(indices), batch_size):
            batch_idx = indices[start:start + batch_size]
            yield X[batch_idx], y[batch_idx]
    else:
        raise ValueError(f'Strategi tidak dikenal: {strategy}')


def accuracy_score(y_true: Array, probs: Array, threshold: float = 0.5) -> float:
    preds = predict_classes(probs, threshold)
    return float(np.mean(preds == y_true))


def compute_classification_metrics(
    y_true: Array, probs: Array, threshold: float = 0.5
) -> Dict[str, float]:
    preds = predict_classes(probs, threshold)
    y_true = y_true.astype(int)
    preds = preds.astype(int)

    tp = float(np.sum((preds == 1) & (y_true == 1)))
    tn = float(np.sum((preds == 0) & (y_true == 0)))
    fp = float(np.sum((preds == 1) & (y_true == 0)))
    fn = float(np.sum((preds == 0) & (y_true == 1)))

    eps = 1e-12
    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)
    accuracy = (tp + tn) / (tp + tn + fp + fn + eps)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


def train_network(
    X: Array,
    y: Array,
    layer_sizes: List[int],
    hidden_activation: str,
    output_activation: str = 'sigmoid',
    strategy: str = 'batch',
    learning_rate: float = 0.01,
    epochs: int = 200,
    batch_size: int = 32,
    random_state: int = 42,
    X_val: Optional[Array] = None,
    y_val: Optional[Array] = None,
) -> Tuple[List[Dict[str, Array]], Dict[str, List[float]], List[str]]:
    rng = np.random.default_rng(random_state)
    params = initialize_parameters(layer_sizes, rng)
    activation_sequence = [hidden_activation] * (len(layer_sizes) - 2) + [output_activation]

    history: Dict[str, List[float]] = {
        'train_loss': [],
        'train_accuracy': [],
    }
    if X_val is not None and y_val is not None:
        history['val_loss'] = []
        history['val_accuracy'] = []

    for epoch in range(epochs):
        for X_batch, y_batch in iterate_batches(X, y, strategy, batch_size, rng):
            probs, caches = forward_pass(X_batch, params, activation_sequence)
            grads = backward_pass(y_batch, caches, params, activation_sequence)
            update_parameters(params, grads, learning_rate)

        train_probs = predict_proba(X, params, activation_sequence)
        history['train_loss'].append(binary_cross_entropy(y, train_probs))
        history['train_accuracy'].append(accuracy_score(y, train_probs))

        if X_val is not None and y_val is not None:
            val_probs = predict_proba(X_val, params, activation_sequence)
            history['val_loss'].append(binary_cross_entropy(y_val, val_probs))
            history['val_accuracy'].append(accuracy_score(y_val, val_probs))

    return params, history, activation_sequence


## Eksperimen: Kombinasi Aktivasi dan Strategi Gradient DescentEksperimen dilakukan pada arsitektur jaringan `[input_dim, 32, 16, 1]` dengan laju belajar konstan. Kombinasi yang diuji:* Fungsi aktivasi hidden layer: **Sigmoid**, **Tanh**, **ReLU**.* Strategi gradient descent: **Batch**, **Mini-batch (32)**, dan **Stochastic**.Metode evaluasi menggunakan akurasi, presisi, recall, dan F1-score pada data uji.

In [ ]:
layer_sizes = [input_dim, 32, 16, 1]
activations_to_test = ['sigmoid', 'tanh', 'relu']
strategies = {
    'batch': None,
    'mini-batch': 32,
    'stochastic': 1,
}

learning_rate = 0.01
epochs = 200

experiment_results: List[Dict[str, float]] = []
training_histories: Dict[Tuple[str, str], Dict[str, List[float]]] = {}
trained_models: Dict[Tuple[str, str], Tuple[List[Dict[str, Array]], List[str]]] = {}

for activation_name in activations_to_test:
    for strategy_name, batch_value in strategies.items():
        batch_size = 1 if strategy_name == 'stochastic' else (batch_value or X_train.shape[0])
        params, history, activation_sequence = train_network(
            X_train,
            y_train,
            layer_sizes,
            hidden_activation=activation_name,
            strategy='stochastic' if strategy_name == 'stochastic' else strategy_name,
            learning_rate=learning_rate,
            epochs=epochs,
            batch_size=batch_size if strategy_name != 'batch' else X_train.shape[0],
            random_state=42,
        )

        train_probs = predict_proba(X_train, params, activation_sequence)
        test_probs = predict_proba(X_test, params, activation_sequence)

        train_loss = binary_cross_entropy(y_train, train_probs)
        test_loss = binary_cross_entropy(y_test, test_probs)
        test_metrics = compute_classification_metrics(y_test, test_probs)

        experiment_results.append({
            'hidden_activation': activation_name,
            'gradient_strategy': strategy_name,
            'learning_rate': learning_rate,
            'epochs': epochs,
            'batch_size': batch_size,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'test_accuracy': test_metrics['accuracy'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall'],
            'test_f1': test_metrics['f1'],
        })

        training_histories[(activation_name, strategy_name)] = history
        trained_models[(activation_name, strategy_name)] = (params, activation_sequence)

        print(
            f"Aktivasi: {activation_name:<7} | Strategi: {strategy_name:<10} | "
            f"Train Loss: {train_loss:.4f} | Test F1: {test_metrics['f1']:.4f}"
        )

results_df = pd.DataFrame(experiment_results)
results_df.sort_values('test_f1', ascending=False).reset_index(drop=True)


## Ringkasan EvaluasiTabel berikut membandingkan performa setiap kombinasi fungsi aktivasi dan strategi gradient descent pada data uji.

In [ ]:
results_df.sort_values('test_f1', ascending=False)


## Perilaku TrainingGrafik berikut memperlihatkan dinamika loss dan akurasi selama proses training untuk beberapa kombinasi terpilih.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

selected_combinations = [
    ('sigmoid', 'batch'),
    ('tanh', 'mini-batch'),
    ('relu', 'stochastic'),
]

for activation_name, strategy_name in selected_combinations:
    history = training_histories.get((activation_name, strategy_name))
    if history is None:
        continue
    axes[0].plot(history['train_loss'], label=f'{activation_name} / {strategy_name}')
    axes[1].plot(history['train_accuracy'], label=f'{activation_name} / {strategy_name}')

axes[0].set_title('Training Loss per Epoch')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()

axes[1].set_title('Training Accuracy per Epoch')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()


## Interpretasi* **Arsitektur**: Model tiga lapis `[input_dim, 32, 16, 1]` memberikan kapasitas yang cukup untuk mempelajari pola pada fitur numerik dan kategorikal yang telah diencoding.* **Aktivasi**: Sigmoid dan Tanh cenderung stabil pada dataset yang tidak terlalu besar, sementara ReLU dapat mengalami konvergensi yang lebih cepat namun memerlukan perhatian terhadap laju belajar untuk menghindari neuron mati.* **Strategi Gradient Descent**: Mini-batch sering kali memberikan kompromi terbaik antara stabilitas gradien (batch) dan kecepatan pembaruan (stochastic). Pengamatan pada loss/akurasi membantu memahami dinamika training masing-masing strategi.* **Metode Evaluasi**: Karena dataset tidak seimbang, F1-score dan recall penting untuk memonitor performa model mendeteksi kasus stroke yang jarang.Eksperimen lanjutan dapat mencakup penyesuaian laju belajar adaptif, penyeimbangan kelas, atau penambahan regularisasi untuk meningkatkan generalisasi.